In [29]:
#Read the csv file
import pandas as pd
df = pd.read_csv('ibb_planungsraeume.csv')
df_bez=pd.read_csv('ibb_bezirke_cleaned.csv')
df

,district_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year
0,1100101,Stülerstraße,22.81,20,22.33,2024
1,1100102,Großer Tiergarten,21.99,45,21.62,2024
2,1100103,Lützowstraße,21.56,57,21,2024
3,1100104,Körnerstraße,22.96,39,22.68,2024
4,1100205,Wilhelmstraße,24.24,84,23.83,2024
...,...,...,...,...,...,...
7041,12601032,Rollbergesiedlung,4.65,164,4.83,2012
7042,12601133,Treuenbrietzener Straße,5.65,136,5.6,2012
7043,12601134,Märkisches Zentrum,5.42,75,5.54,2012
7044,12601235,Dannenwalder Weg,5.32,137,5.39,2012


In [30]:
df_bez

,district_id,neighborhood,median_net_rent,number_of_cases,mean_net_rent,year
0,1,Mitte,19.91,4169,19.91,2024
1,2,Friedrichshain-Kreuzberg,19.42,2509,19.26,2024
2,3,Pankow,17.00,3636,17.65,2024
3,4,Charlottenburg-Wilmersdorf,19.39,3036,19.40,2024
4,5,Spandau,12.00,2403,13.32,2024
...,...,...,...,...,...,...
152,8,Neukölln,6.79,4027,7.02,2012
153,9,Treptow-Köpenick,6.50,4883,6.76,2012
154,10,Marzahn-Hellersdorf,5.17,3834,5.56,2012
155,11,Lichtenberg,6.50,2898,6.65,2012


In [31]:
# Created new column called street_id, set a random value
df["street_id"] = 1100103
df

,district_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id
0,1100101,Stülerstraße,22.81,20,22.33,2024,1100103
1,1100102,Großer Tiergarten,21.99,45,21.62,2024,1100103
2,1100103,Lützowstraße,21.56,57,21,2024,1100103
3,1100104,Körnerstraße,22.96,39,22.68,2024,1100103
4,1100205,Wilhelmstraße,24.24,84,23.83,2024,1100103
...,...,...,...,...,...,...,...
7041,12601032,Rollbergesiedlung,4.65,164,4.83,2012,1100103
7042,12601133,Treuenbrietzener Straße,5.65,136,5.6,2012,1100103
7043,12601134,Märkisches Zentrum,5.42,75,5.54,2012,1100103
7044,12601235,Dannenwalder Weg,5.32,137,5.39,2012,1100103


In [32]:
# Take first 2 digits,discarding rest 6 digits
#moving district id to street id and making district id unique
df['street_id'] = df['district_id']
df['district_id'] = df['district_id'].astype(str).str[:-6]
df

,district_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id
0,1,Stülerstraße,22.81,20,22.33,2024,1100101
1,1,Großer Tiergarten,21.99,45,21.62,2024,1100102
2,1,Lützowstraße,21.56,57,21,2024,1100103
3,1,Körnerstraße,22.96,39,22.68,2024,1100104
4,1,Wilhelmstraße,24.24,84,23.83,2024,1100205
...,...,...,...,...,...,...,...
7041,12,Rollbergesiedlung,4.65,164,4.83,2012,12601032
7042,12,Treuenbrietzener Straße,5.65,136,5.6,2012,12601133
7043,12,Märkisches Zentrum,5.42,75,5.54,2012,12601134
7044,12,Dannenwalder Weg,5.32,137,5.39,2012,12601235


In [33]:
# Converted Object type in to Integer type for joining tables and to match other table columns data type
df['district_id'] = df['district_id'].astype('int64')

In [36]:
#Merging Bezirke subset table into Plannugsraüme table
df_bez_subset = df_bez[["district_id", "neighborhood"]]
df=pd.merge(df,df_bez_subset, on='district_id')
df=df.drop_duplicates()
df

,district_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id,neighborhood_x,neighborhood_y
0,1,Stülerstraße,22.81,20,22.33,2024,1100101,Mitte,Mitte
13,1,Großer Tiergarten,21.99,45,21.62,2024,1100102,Mitte,Mitte
26,1,Lützowstraße,21.56,57,21,2024,1100103,Mitte,Mitte
39,1,Körnerstraße,22.96,39,22.68,2024,1100104,Mitte,Mitte
52,1,Wilhelmstraße,24.24,84,23.83,2024,1100205,Mitte,Mitte
...,...,...,...,...,...,...,...,...,...
91533,12,Rollbergesiedlung,4.65,164,4.83,2012,12601032,Reinickendorf,Reinickendorf
91546,12,Treuenbrietzener Straße,5.65,136,5.6,2012,12601133,Reinickendorf,Reinickendorf
91559,12,Märkisches Zentrum,5.42,75,5.54,2012,12601134,Reinickendorf,Reinickendorf
91572,12,Dannenwalder Weg,5.32,137,5.39,2012,12601235,Reinickendorf,Reinickendorf


In [37]:
# Renaming the column
df.rename(columns={'district_id':'neighborhood_id'}, inplace=True)
df

,neighborhood_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id,neighborhood_x,neighborhood_y
0,1,Stülerstraße,22.81,20,22.33,2024,1100101,Mitte,Mitte
13,1,Großer Tiergarten,21.99,45,21.62,2024,1100102,Mitte,Mitte
26,1,Lützowstraße,21.56,57,21,2024,1100103,Mitte,Mitte
39,1,Körnerstraße,22.96,39,22.68,2024,1100104,Mitte,Mitte
52,1,Wilhelmstraße,24.24,84,23.83,2024,1100205,Mitte,Mitte
...,...,...,...,...,...,...,...,...,...
91533,12,Rollbergesiedlung,4.65,164,4.83,2012,12601032,Reinickendorf,Reinickendorf
91546,12,Treuenbrietzener Straße,5.65,136,5.6,2012,12601133,Reinickendorf,Reinickendorf
91559,12,Märkisches Zentrum,5.42,75,5.54,2012,12601134,Reinickendorf,Reinickendorf
91572,12,Dannenwalder Weg,5.32,137,5.39,2012,12601235,Reinickendorf,Reinickendorf


In [38]:
#Replacing kein daten with nan
df[["median_net_rent", "number_of_cases","mean_net_rent"]] = df[["median_net_rent", "number_of_cases","mean_net_rent"]].replace({
    "keine Daten": "np.nan"
})
df

,neighborhood_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id,neighborhood_x,neighborhood_y
0,1,Stülerstraße,22.81,20,22.33,2024,1100101,Mitte,Mitte
13,1,Großer Tiergarten,21.99,45,21.62,2024,1100102,Mitte,Mitte
26,1,Lützowstraße,21.56,57,21,2024,1100103,Mitte,Mitte
39,1,Körnerstraße,22.96,39,22.68,2024,1100104,Mitte,Mitte
52,1,Wilhelmstraße,24.24,84,23.83,2024,1100205,Mitte,Mitte
...,...,...,...,...,...,...,...,...,...
91533,12,Rollbergesiedlung,4.65,164,4.83,2012,12601032,Reinickendorf,Reinickendorf
91546,12,Treuenbrietzener Straße,5.65,136,5.6,2012,12601133,Reinickendorf,Reinickendorf
91559,12,Märkisches Zentrum,5.42,75,5.54,2012,12601134,Reinickendorf,Reinickendorf
91572,12,Dannenwalder Weg,5.32,137,5.39,2012,12601235,Reinickendorf,Reinickendorf


In [39]:
#Writing to csv file
df.to_csv('ibb_planungsraeume_cleaned.csv', index=False)
df

,neighborhood_id,street_name,median_net_rent,number_of_cases,mean_net_rent,year,street_id,neighborhood_x,neighborhood_y
0,1,Stülerstraße,22.81,20,22.33,2024,1100101,Mitte,Mitte
13,1,Großer Tiergarten,21.99,45,21.62,2024,1100102,Mitte,Mitte
26,1,Lützowstraße,21.56,57,21,2024,1100103,Mitte,Mitte
39,1,Körnerstraße,22.96,39,22.68,2024,1100104,Mitte,Mitte
52,1,Wilhelmstraße,24.24,84,23.83,2024,1100205,Mitte,Mitte
...,...,...,...,...,...,...,...,...,...
91533,12,Rollbergesiedlung,4.65,164,4.83,2012,12601032,Reinickendorf,Reinickendorf
91546,12,Treuenbrietzener Straße,5.65,136,5.6,2012,12601133,Reinickendorf,Reinickendorf
91559,12,Märkisches Zentrum,5.42,75,5.54,2012,12601134,Reinickendorf,Reinickendorf
91572,12,Dannenwalder Weg,5.32,137,5.39,2012,12601235,Reinickendorf,Reinickendorf
